In [19]:
%load_ext autoreload
%autoreload 2
from sklearn.model_selection import ParameterGrid
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.linear_model import LogisticRegression
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn import metrics
from notebooks.sentiment.util import eval
from sentiment.new_data import InterTASSAugmented
from sentiment.tass import InterTASSReader
from tqdm import tqdm

def preprocess_tweets(str_):
    mentions = r'(?:@[^\s]+)'
    urls = r'(?:https?\://t.co/[\w]+)'
    str_ = re.sub(mentions, '', str_)
    return re.sub(urls, '', str_)

param_grid = {
    'clf__random_state': [0],
    'clf__penalty': ('l1', 'l2'),
    'clf__C': (0.05, 0.1, 0.3, 0.5),
}

vect = TfidfVectorizer(tokenizer=word_tokenize,
                       binary=True,
                       analyzer="char_wb",
                       ngram_range=(1, 6),
                       min_df=5,
                       max_df=0.95,
                       preprocessor=preprocess_tweets,
                       stop_words=stopwords.words("spanish"))

vect2 = TfidfVectorizer(tokenizer=word_tokenize,
                        binary=True,
                        analyzer="word",
                        ngram_range=(1, 5),
                        preprocessor=preprocess_tweets,
                        stop_words=stopwords.words("spanish"))

params_list = list(ParameterGrid(param_grid))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
pipeline = Pipeline([
            ('feats', FeatureUnion([
                    ('vect', vect),  # can pass in either a pipeline
                    ('vect1', vect2),  # or a transformer
                ])),
            ('clf', LogisticRegression()),
        ])
# reader = InterTASSAugmented(ratio=0.9)  # Class to use augmented data
# X_train, y_train = reader.Xy()
train = "../intertass-ES-train-tagged.xml"
train = InterTASSReader(train)
X_train, y_train = list(train.X()), list(train.y())


corpus = "../intertass-ES-development-tagged.xml"
dev = InterTASSReader(corpus)
X_dev, y_dev = list(dev.X()), list(dev.y())

results = []
for params in tqdm(params_list):
    pipeline.set_params(**params)
    pipeline.fit(X_train, y_train)
    result = eval(pipeline, X_dev, y_dev)
    
    results.append({
        **result,
        **params,
    })

100%|██████████| 8/8 [00:06<00:00,  1.33it/s]


In [21]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df.sort_values(['acc', 'f1'], ascending=False)[:10]

,acc,clf__C,clf__penalty,clf__random_state,f1
7,0.565217,0.50,l2,0,0.318593
5,0.555336,0.30,l2,0,0.309252
6,0.505929,0.50,l1,0,0.260154
3,0.468379,0.10,l2,0,0.208502
1,0.446640,0.05,l2,0,0.173979
0,0.432806,0.05,l1,0,0.151034
2,0.432806,0.10,l1,0,0.151034
4,0.432806,0.30,l1,0,0.151034
